In [ ]:
import requests, json, pprint
import pandas as pd
import numpy as np
from jsonpath_ng import jsonpath, parse
import datetime

In [1]:
pip install jsonpath_ng

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 10.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 26.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 32.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#1.load input 
timestamp_csv = pd.read_csv ('timestamp1.csv')
coordinate_csv = pd.read_csv('coordinate.csv')
# API_KEY = "69e792e1189be59d0196f827035b39cb" #alternative api key not yet activated
API_KEY = "d8a7f0f4a549a5083e601317b765430f"
units = "metric"

unixtodatetime_list=[]
datetime_list=[]
temp_list=[]
humidity_list=[]

k=0

In [ ]:
while k < len(coordinate_csv. index):

    location = coordinate_csv.iloc[k]['Name']
    lat = str(coordinate_csv.iloc[k]['lat'])
    lon = str(coordinate_csv.iloc[k]['lon'])
    
    print(location)
    print(lat)
    print(lon)
    
    i=0

    while i < len(timestamp_csv. index):  #TO-DO, if x datapoint to request, i< x.

        
        dt = (timestamp_csv.loc[i].at["Timestamp"])
    
        #2.API Request & Response

        URL = "https://api.openweathermap.org/data/3.0/onecall/timemachine?" + "lat=" + lat + "&lon=" + lon + "&dt=" + str(dt) + "&appid=" + API_KEY + "&units=" + units
        response = requests.get(URL)

        # checking the status code of the request
        
        if response.status_code == 200:
        # getting data in the json format
            data = response.json()
         #getting required info
            uplat = parse("$.lat").find(data)[0].value
            uplon = parse("$.lon").find(data)[0].value
            unix_timestamp= parse("$.data[*].dt").find(data)[0].value
            temperature = parse("$.data[*].temp").find(data)[0].value
            humidity = parse("$.data[*].humidity").find(data)[0].value
            dt_datetime = datetime.datetime.fromtimestamp(dt, datetime.timezone(datetime.timedelta(hours=8)))
            

                # print(f"lat: {uplat}")
                # print(f"lon: {uplon}")
                # print(f"datetime: {datetime}")
                # print(f"Temperature: {temperature}")
                # print(f"Humidity: {humidity}")

            i=i+1

        else:
            # showing the error message
            print("Error in the HTTP request")

        #3. Write data into dataframe
        unixtodatetime_list.append(dt_datetime)
        datetime_list.append(datetime)
        temp_list.append(temperature)
        humidity_list.append(humidity)

    else:
        d = {'timestamp':unix_timestamp,'datetime':unixtodatetime_list,'temperature':temp_list,'humidity':humidity_list}
        df = pd.DataFrame(d)
        df.to_csv(location + '.csv')
        print("Location:" + location + " done.\n")
        
        #Reset the dataframe for next iteration
        unixtodatetime_list=[]
        datetime_list=[]
        temp_list=[]
        humidity_list=[]
        
    k=k+1
    
else:
    print("All data logged succesfully, check the output.")

    

In [ ]:
dt